In [1]:
!pip install scikit-image

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 82.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 43.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 81.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.4/315.4 KB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 115.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.2/228.2 KB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 112.8 MB/s eta 0:00:0000:01


In [1]:
import sys
import os

# Define the base path
base_path = './tibo1/Lab3/'
# Set the working directory to the project root
#os.chdir('./tibo1/Lab3/')

# Convert to absolute path and add to sys.path
sys.path.append(os.path.abspath(base_path))

In [2]:
import os
import gzip
import pickle
import numpy as np
import cv2
from skimage import filters
import torch
from typing import Tuple, List
from tqdm import tqdm
from utils import uint2single, single2tensor4, test_onesplit, tensor2uint, get_device

### 1. Pre-Definitions
#### 1a. Loader Function for the data

In [3]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
    return loaded_object

#### 1b. Utility Functions

In [4]:
from matplotlib import pyplot as plt
def pad_image(image):
    ## Ensure image is a proper numpy array
    #if not isinstance(image, np.ndarray):
    #    raise ValueError("Input must be a numpy array.")
#
    ## Ensure image is contiguous in memory
    #if not image.flags['C_CONTIGUOUS']:
    #    image = np.ascontiguousarray(image)
#
    ## Ensure image has correct dtype
    #image = image.astype(np.uint8)
#
    ## Check dimensions
    #if len(image.shape) != 2:
    #    raise ValueError(f"Image must be grayscale (2D). Got shape: {image.shape}")

    # Compute padding
    h, w = image.shape
    target_size = max(h, w)
    padded_image = cv2.copyMakeBorder(
        image,
        0, target_size - h,
        0, target_size - w,
        cv2.BORDER_CONSTANT,
        value=0
    )

    return padded_image

def heal_image(img: np.ndarray):
    # Function to remove small artifacts from the image
    assert len(img.shape) == 2, "Image must be grayscale"
    height, width = img.shape
    ret, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    min_patch_size = 30
    for contour in contours:
        contour_area = cv2.contourArea(contour)
        if contour_area < min_patch_size:
            mask = np.zeros((height, width), np.uint8)
            cv2.drawContours(mask, [contour], 0, 255, -1)
            img = cv2.inpaint(img, mask, 2, cv2.INPAINT_TELEA)
    return img

def denoise_enhance_image( image: np.ndarray, model, noise_level_model_tensor, device, denoise_before: bool) -> np.ndarray:
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(4, 4))
    if denoise_before:
        image = denoise_image(image, model, noise_level_model_tensor, device)
        image = clahe.apply(image)
    else:
        image = clahe.apply(image)
        image = denoise_image(image, model, noise_level_model_tensor, device)
    image = heal_image(image)
    return image


def process_image( image: np.ndarray, target_size: Tuple[int, int], model, noise_level_model_tensor, device, is_test: bool = False, denoise_before: bool = False) -> np.ndarray:
    assert len(image.shape) == 2, 'Image must be grayscale'
    # Pad image to create square aspect ratio
    padded_image = pad_image(image)
    # TODO SHOULD WE ALSO DENOISE TEST OR NOT
    if not is_test:
        padded_image = denoise_enhance_image(
            padded_image, model, noise_level_model_tensor, device, denoise_before
        )
    # Resize image to target size
    resized = cv2.resize(padded_image, target_size, interpolation=cv2.INTER_LINEAR)
    return resized

def process_mask(mask: np.ndarray, target_size: Tuple[int, int]) -> np.ndarray:
    assert len(mask.shape) == 2, 'Mask must be grayscale'
    # Pad image to create square aspect ratio
    padded_mask = pad_image(mask)
    # Apply bilateral filter
    padded_mask = cv2.bilateralFilter(padded_mask, d=9, sigmaColor=1, sigmaSpace=1)
    # Threshold the filtered image
    threshold_value = filters.threshold_otsu(padded_mask)
    padded_mask = (padded_mask > threshold_value * 2).astype(np.uint8) * 255  # Threshold
    # Dilate the filtered image
    kernel = np.ones((2, 2), np.uint8)
    padded_mask = cv2.dilate(padded_mask, kernel, iterations=1)
    # Apply Gaussian blur to the dilated and filtered image
    padded_mask = cv2.GaussianBlur(padded_mask, (3, 3), 2)
    padded_mask = (padded_mask > threshold_value).astype(np.uint8) * 255
    # Resize image to target size
    resized = cv2.resize(padded_mask, target_size, interpolation=cv2.INTER_LINEAR)
    return resized


#### 1c. Load DRUNet model

In [5]:
# Load the pre-trained DRUNet model
from drunet.drunet import UNetRes

def load_drunet_model(model_path: str, device: str = 'cpu'):
    print("Running on device: ", device)
    n_channels = 1
    noise_level_model = 5
    noise_level_model_tensor = torch.FloatTensor([noise_level_model / 255.]).to(device)

    model = UNetRes(
        in_nc=n_channels + 1,
        out_nc=n_channels,
        nc=[64, 128, 256, 512],
        nb=4,
        act_mode='R',
        downsample_mode="strideconv",
        upsample_mode="convtranspose"
    )
    model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True), strict=True)
    model.eval()
    for param in model.parameters():
        param.requires_grad = False
    model = model.to(device)
    return model, noise_level_model_tensor

def denoise_image(image: np.ndarray, model, noise_level_model_tensor, device) -> np.ndarray:
    image = np.expand_dims(image, axis=2)
    image = uint2single(image)
    image = single2tensor4(image)

    # Ensure the noise_level_model_tensor is on the same device as the input
    noise_level_model_tensor = noise_level_model_tensor.to(device)
    image = image.to(device)  # Move image to the correct device

    image = torch.cat((
        image,
        noise_level_model_tensor.repeat(1, 1, image.shape[2], image.shape[3])
    ), dim=1)
    image = image.to(device)
    denoised_img = test_onesplit(model, image, refield=32)
    denoised_img = tensor2uint(denoised_img)
    return denoised_img

### 2. Preprocess Data
#### 2.a Load Data

In [6]:
# Paths to your data files
train_data_path = './data/train.pkl'
test_data_path = './data/test.pkl'

# Load the data
train_data = load_zipped_pickle(train_data_path)
test_data = load_zipped_pickle(test_data_path)

print(f'Loaded {len(train_data)} patients from training data.')
print(f'Loaded {len(test_data)} patients from test data.')

Loaded 65 patients from training data.
Loaded 20 patients from test data.


#### 2.b Initialize the DRUNet model

In [7]:
# Path to the pretrained DRUNet model
drunet_model_path = './drunet/drunet_gray.pth'
device = get_device()
print(f'Running on device: {device}')

# Load the model
drunet_model, noise_level_model_tensor = load_drunet_model(drunet_model_path, device)
TARGET_SIZE = (224, 224)  # Adjust as needed currently 2 times amateur samples dim

Running on device: mps
Running on device:  mps


#### 2.c Preprocess Training Data
##### Separate Amateur and Expert Data

In [8]:
# Separate amateur and expert data
amateur_data = [patient for patient in train_data if patient['dataset'] == 'amateur']
expert_data = [patient for patient in train_data if patient['dataset'] == 'expert']

print(f'Expert data: {len(expert_data)} patients')
print(f'Amateur data: {len(amateur_data)} patients')

Expert data: 19 patients
Amateur data: 46 patients


In [9]:
def stratified_bootstrap(y_train, frame_indexes):
    num_bootstrap_samples = y_train.shape[0] - len(frame_indexes)
    print(f'Num of bootstrap samples {num_bootstrap_samples}')
    print(f'Frame indices:{frame_indexes}')

    # perform bootstrapping
    bootstrap_mask_indexes = np.random.choice(frame_indexes, size=num_bootstrap_samples, replace=True)
    bootstrap_masks = y_train[bootstrap_mask_indexes]

    return bootstrap_masks, bootstrap_mask_indexes

##### Process Images and Masks

In [16]:
def preprocess_dataset(data: List[dict], target_size: Tuple[int, int], drunet_model, noise_level_model_tensor, device, is_test: bool = False):
    print("Running on device: ", device)
    total_frames = 0

    for i, patient in enumerate(tqdm(data, desc=f'Processing {"Test" if is_test else "Train"} Data: ')):
        video = patient['video']

        if not is_test:
            masks = patient['label']
            frames = patient['frames']

            # Process images
            new_video = [
                process_image(
                    video[..., frame].astype(np.uint8),
                    target_size,
                    drunet_model,
                    noise_level_model_tensor,
                    device,
                    is_test=False,
                    denoise_before=False
                )
                for frame in range(video.shape[-1])
            ]

            # Augmentation: denoise before enhancement
            new_video += [
                process_image(
                    video[..., frame].astype(np.uint8),
                    target_size,
                    drunet_model,
                    noise_level_model_tensor,
                    device,
                    is_test=False,
                    denoise_before=True
                )
                for frame in range(video.shape[-1])
            ]

            new_video = np.array(new_video) / 255.0  # Normalize images


            new_masks = [
                process_mask(masks[..., frame].astype(np.uint8), target_size)
                                  for frame in range(masks.shape[-1])] * 2  # Duplicate masks for augmented images
            new_masks = np.array(new_masks).astype(np.float32)

            # Needed for the bootstrap to sample also from the Augmentation subset
            patient['frame'] = np.concatenate((frames, np.array([frame + video.shape[-1] for frame in frames])), axis=0)
            print("New frame shape after augmentation: ", patient['frame'])
            print("Patient shape after augmentation: ", patient['frame'].shape)

            bootstrapped_masks, bootstrap_indices = stratified_bootstrap(new_masks, frames)
            print("Bootstrapped masks shape: ", bootstrapped_masks.shape)
            print("Bootstrapped indices shape: ", bootstrap_indices.shape)
            print("Number of frames of video before bootstrapping: ", new_video.shape[0])
            print("Number of frames of masks before bootstrapping: ", new_masks.shape[0])
            bootstrapped_images = new_video[bootstrap_indices]

            print("Bootstrapped images shape: ", bootstrapped_images.shape)

            # Concatenate bootstrapped data
            new_masks = np.concatenate((new_masks, bootstrapped_masks), axis=0)
            new_video = np.concatenate((new_video, bootstrapped_images), axis=0)

            # double the video and masks
            new_masks = np.concatenate((new_masks, new_masks), axis=0).astype(np.float32)
            new_video = np.concatenate((new_video, new_video), axis=0).astype(np.float32)

            patient['video'] = new_video
            patient['label'] = new_masks

        else:
            # Test data processing (no masks)
            new_video = [
                process_image(
                    video[..., frame].astype(np.uint8),
                    target_size,
                    drunet_model,
                    noise_level_model_tensor,
                    device,
                    is_test=True,
                    denoise_before=False
                )
                for frame in range(video.shape[-1])
            ]
            new_video = np.array(new_video) / 255.0  # Normalize images

            patient['video'] = new_video

        total_frames += new_video.shape[0]

        if i % 5 == 0:
            print(f"Processed {i+1}/{len(data)} patients. Total frames so far: {total_frames}")

    print("Finished preprocessing and bootstrapping.")
    return data, total_frames

In [17]:
preprocessed_expert_data, expert_total_frames = preprocess_dataset(expert_data, TARGET_SIZE, drunet_model, noise_level_model_tensor, device, is_test=False)
#preprocessed_amateur_data, amateur_total_frames = preprocess_dataset(amateur_data, TARGET_SIZE, drunet_model, noise_level_model_tensor, device, is_test=False)

Running on device:  mps


Processing Train Data:   0%|          | 0/19 [00:00<?, ?it/s]

New frame shape after augmentation:  [ 20  34  68 244 258 292]
Patient shape after augmentation:  (6,)
Num of bootstrap samples 445
Frame indices:[20, 34, 68]
Bootstrapped masks shape:  (445, 224, 224)
Bootstrapped indices shape:  (445,)
Number of frames of video before bootstrapping:  448
Number of frames of masks before bootstrapping:  448
Bootstrapped images shape:  (445, 224, 224)


Processing Train Data:   5%|▌         | 1/19 [03:24<1:01:15, 204.19s/it]

Processed 1/19 patients. Total frames so far: 1786
New frame shape after augmentation:  [  6  20  60  89 103 143]
Patient shape after augmentation:  (6,)
Num of bootstrap samples 163
Frame indices:[6, 20, 60]
Bootstrapped masks shape:  (163, 224, 224)
Bootstrapped indices shape:  (163,)
Number of frames of video before bootstrapping:  166
Number of frames of masks before bootstrapping:  166
Bootstrapped images shape:  (163, 224, 224)


...

Processed 11/19 patients. Total frames so far: 8262
New frame shape after augmentation:  [  2   7  13  97 102 108]
Patient shape after augmentation:  (6,)
Num of bootstrap samples 187
Frame indices:[2, 7, 13]
Bootstrapped masks shape:  (187, 224, 224)
Bootstrapped indices shape:  (187,)
Number of frames of video before bootstrapping:  190
Number of frames of masks before bootstrapping:  190
Bootstrapped images shape:  (187, 224, 224)


Processing Train Data:  63%|██████▎   | 12/19 [27:39<15:44, 134.93s/it]

New frame shape after augmentation:  [  3  30  77 104 131 178]
Patient shape after augmentation:  (6,)
Num of bootstrap samples 199
Frame indices:[3, 30, 77]
Bootstrapped masks shape:  (199, 224, 224)
Bootstrapped indices shape:  (199,)
Number of frames of video before bootstrapping:  202
Number of frames of masks before bootstrapping:  202
Bootstrapped images shape:  (199, 224, 224)


Processing Train Data:  68%|██████▊   | 13/19 [30:42<14:10, 141.72s/it]


KeyboardInterrupt: 

#### 2.e Train-Validation Split for Later

In [23]:
print(f"Starting train/val split. Total expert frames: {expert_total_frames}, ")
            #f"Total amateur frames: {amateur_total_frames}")

def train_val_split(data: list, nbr_total_frames: int, split_ratio=0.8) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:

    train_images, train_masks, val_images, val_masks = [], [], [], []
    train_frames = 0
    train_limit = nbr_total_frames * split_ratio  # Precompute the training frame limit

    print(f"Starting train/val split. Total frames: {nbr_total_frames}. Train split ratio: {split_ratio:.2f}")
    for patient in tqdm(data, desc="Splitting patients into train/val"):
        if train_frames < train_limit:
            train_images.append(patient['video'])
            train_masks.append(patient['label'])
            train_frames += patient['video'].shape[0]
        else:
            val_images.append(patient['video'])
            val_masks.append(patient['label'])

    # Combine lists into arrays
    train_images = np.concatenate(train_images, axis=0)
    train_masks = np.concatenate(train_masks, axis=0)
    val_images = np.concatenate(val_images, axis=0)
    val_masks = np.concatenate(val_masks, axis=0)

    print(f"Finished train/val split: Train frames: {train_images.shape[0]}, Val frames: {val_images.shape[0]}")
    return train_images, train_masks, val_images, val_masks

Starting train/val split. Total expert frames: 13478, 


In [24]:
expert_train_images, expert_train_masks, expert_val_images, expert_val_masks = train_val_split(preprocessed_expert_data, expert_total_frames)
#amateur_train_images, amateur_train_masks, amateur_val_images, amateur_val_masks = train_val_split(preprocessed_amateur_data, amateur_total_frames)

Starting train/val split. Total frames: 13478. Train split ratio: 0.80


Splitting patients into train/val: 100%|██████████| 19/19 [00:00<00:00, 228999.36it/s]


Finished train/val split: Train frames: 10798, Val frames: 2680


#### 2.f Save Preprocessed Data

In [25]:
# Save expert data
np.save('./tibo1/Lab3/out/preprocessed/expert_X_train.npy', expert_train_images)
np.save('./tibo1/Lab3/out/preprocessed/expert_y_train.npy', expert_train_masks)
np.save('./tibo1/Lab3/out/preprocessed/expert_X_val.npy', expert_val_images)
np.save('./tibo1/Lab3/out/preprocessed/expert_y_val.npy', expert_val_masks)

# Save amateur data
# np.save('./tibo1/Lab3/out/preprocessed/amateur_X_train.npy', amateur_train_images)
# np.save('./tibo1/Lab3/out/preprocessed/amateur_y_train.npy', amateur_train_masks)
# np.save('./tibo1/Lab3/out/preprocessed/amateur_X_val.npy', amateur_val_images)
# np.save('./tibo1/Lab3/out/preprocessed/amateur_y_val.npy', amateur_val_masks)

#### 2g. Preprocess test data

| Step                  | Train/Validation Dataset           | Test Dataset                        |
|-----------------------|-------------------------------------|--------------------------------------|
| Padding               | Yes                                | Yes                                 |
| Resizing              | Yes                                | Yes                                 |
| Augmentation          | Yes (e.g., rotation, affine, etc.) | No                                  |
| Bootstrapping         | Yes                                | No                                  |
| Denoising/Enhancement | Yes                                | Yes                                 |
| Data Duplication      | Yes                                | No                                  |


In [13]:
test_images, _ = preprocess_dataset(test_data, TARGET_SIZE, drunet_model, noise_level_model_tensor, device, is_test=True)

# Save test data
np.save('./tibo1/Lab3/out/preprocessed/test.npy', test_images)

Running on device:  cuda


Processing Test Data:   5%|▌         | 1/20 [00:06<02:03,  6.51s/it]

Processed 1/20 patients. Total frames so far: 224


Processing Test Data:  30%|███       | 6/20 [00:38<01:30,  6.49s/it]

Processed 6/20 patients. Total frames so far: 1344


...

Processed 11/20 patients. Total frames so far: 2464


Processing Test Data:  80%|████████  | 16/20 [01:42<00:25,  6.47s/it]

Processed 16/20 patients. Total frames so far: 3584


Processing Test Data: 100%|██████████| 20/20 [02:08<00:00,  6.40s/it]


Finished preprocessing and bootstrapping.


In [ ]:
drunet_model.detach()
if get_device() == 'cuda':
    torch.cuda.empty_cache()
elif get_device() == 'mps':
    os.system('mps free')
else:
    print("No action taken.")